Telepítsd a https://bellard.org/ts_server/ -t, konfiguráld, töldtsd le a megfelelő modellt, a munkafüzetben frissítsd az IP-címet a saját környezetednek megfelelően.

In [3]:
!pip install datasets

In [5]:
from datasets import load_dataset
import json
import os

# Adatkészlet betöltése
dataset = load_dataset('yahma/alpaca-cleaned')

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 51760
    })
})


In [ ]:
from datasets import load_dataset
import requests
import json
import time
import os

# Adatkészlet betöltése
dataset = load_dataset('yahma/alpaca-cleaned')

# Fordító beállítások
url = "http://192.168.7.79:8080"
api_key = ""  # Add your API key here
model = "madlad400_7b"
source_lang = "en"
target_lang = "hu"


# Ideiglenes könyvtár a chunkok mentéséhez
temp_dir = "temp_chunks"
os.makedirs(temp_dir, exist_ok=True)

progress_file = os.path.join(temp_dir, "progress.json")

def make_request(path, json):
    response = requests.post(url + path, headers={"Authorization": "Bearer " + api_key}, json=json)
    return response

def translate_text(text, source_lang, target_lang):
    req = {"text": [text], "source_lang": source_lang, "target_lang": target_lang}
    result = make_request("/v1/engines/" + model + "/translate", req)
    if result.status_code != 200:
        print("Request error:", result.text)
        return None
    resp = result.json()
    translation = resp["translations"][0]["text"]
    return translation

# Ellenőrizzük, hogy van-e mentett progressz és folytatjuk onnan
start_idx = 0
if os.path.exists(progress_file):
    with open(progress_file, 'r', encoding='utf-8') as f:
        progress_data = json.load(f)
        start_idx = progress_data['last_processed_idx'] + 1

# Fordítási folyamat és mentés chunkokban
chunk_size = 10
translated_data = []
total_rows = len(dataset['train'])
start_time = time.time()
time_per_chunk = []

for idx in range(start_idx, total_rows):
    row = dataset['train'][idx]
    translated_row = {
        'output': translate_text(row['output'], source_lang, target_lang),
        'input': translate_text(row['input'], source_lang, target_lang),
        'instruction': translate_text(row['instruction'], source_lang, target_lang)
    }
    translated_data.append(translated_row)

    # Mentés és progress kijelzés
    if (idx + 1) % chunk_size == 0 or idx + 1 == total_rows:
        chunk_file = os.path.join(temp_dir, f'translated_chunk_{idx + 1}.json')
        with open(chunk_file, 'w', encoding='utf-8') as f:
            json.dump(translated_data, f, ensure_ascii=False, indent=4)
        translated_data = []  # Chunk kiürítése

        # Mentett progressz frissítése
        with open(progress_file, 'w', encoding='utf-8') as f:
            json.dump({'last_processed_idx': idx}, f)

        progress = (idx + 1) / total_rows * 100
        elapsed_time = time.time() - start_time
        time_per_chunk.append(elapsed_time / ((idx + 1 - start_idx) // chunk_size + 1))
        avg_time_per_chunk = sum(time_per_chunk) / len(time_per_chunk)
        remaining_chunks = (total_rows - (idx + 1)) / chunk_size
        remaining_time = remaining_chunks * avg_time_per_chunk

        print(f"Translation progress: {progress:.2f}%")
        print(f"Estimated remaining time: {remaining_time / 60:.2f} minutes")

        time.sleep(0.1)  # Kis szünet a jobb olvashatóság érdekében

# Chunkok egyesítése egy végleges JSON fájlba
final_translations = []
for chunk_file in sorted(os.listdir(temp_dir)):
    if chunk_file.startswith("translated_chunk_"):
        with open(os.path.join(temp_dir, chunk_file), 'r', encoding='utf-8') as f:
            chunk_data = json.load(f)
            final_translations.extend(chunk_data)

with open('translated_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(final_translations, f, ensure_ascii=False, indent=4)

print("Translation completed and saved to translated_dataset.json")

# Ideiglenes progressz fájl törlése
os.remove(progress_file)

Translation progress: 0.02%
Estimated remaining time: 974.24 minutes
Translation progress: 0.04%
Estimated remaining time: 1109.77 minutes
Translation progress: 0.06%
Estimated remaining time: 1197.30 minutes
Translation progress: 0.08%
Estimated remaining time: 1263.26 minutes
Translation progress: 0.10%
Estimated remaining time: 1317.88 minutes
Translation progress: 0.12%
Estimated remaining time: 1342.48 minutes
Translation progress: 0.14%
Estimated remaining time: 1357.98 minutes
Translation progress: 0.15%
Estimated remaining time: 1369.80 minutes
Translation progress: 0.17%
Estimated remaining time: 1386.93 minutes
Translation progress: 0.19%
Estimated remaining time: 1400.79 minutes
Translation progress: 0.21%
Estimated remaining time: 1410.82 minutes
Translation progress: 0.23%
Estimated remaining time: 1413.64 minutes
Translation progress: 0.25%
Estimated remaining time: 1413.28 minutes
Translation progress: 0.27%
Estimated remaining time: 1412.47 minutes
Translation progress: